Crop into QLD

In [16]:
import os
import json
import xarray as xr
import geopandas as gpd
import rioxarray
import rasterio.features
import numpy as np

# === Paths ===
shapefile_path = r"Data\QLD Bio Regions\QLD_BioRegion_Shape_Files\Biogeographic_Regions.shp"
input_nc_folder = r"Data - Modelling\Annual_DB\1. AUS\Radiation"
output_nc_folder = r"Data - Modelling\Annual_DB\2. QLD_Cropped\Radiation"
os.makedirs(output_nc_folder, exist_ok=True)

# === Load shapefile and assign integer codes ===
qld_shape = gpd.read_file(shapefile_path).to_crs("EPSG:4326")
region_list = qld_shape["q_reg"].tolist()
region_code_map = {code: idx for idx, code in enumerate(region_list)}
reverse_map = {idx: code for code, idx in region_code_map.items()}  # for JSON output

# === Add integer codes to GeoDataFrame ===
qld_shape["region_id"] = qld_shape["q_reg"].map(region_code_map)

# === List .nc files ===
nc_files = [f for f in os.listdir(input_nc_folder) if f.endswith(".nc")]

# === Process each file ===
for nc_file in nc_files:
    nc_path = os.path.join(input_nc_folder, nc_file)
    print(f"\n📂 Processing: {nc_file}")

    try:
        ds = xr.open_dataset(nc_path)
        ds = ds.rio.write_crs("EPSG:4326", inplace=True)

        # === Clip dataset to Queensland bounding geometry ===
        clipped = ds.rio.clip(qld_shape.geometry, qld_shape.crs, drop=True)

        # === Prepare rasterization ===
        transform = clipped.rio.transform()
        out_shape = (clipped.sizes["lat"], clipped.sizes["lon"])

        shapes = [
            (geom, rid) for geom, rid in zip(qld_shape.geometry, qld_shape.region_id)
        ]
        region_raster = rasterio.features.rasterize(
            shapes=shapes,
            out_shape=out_shape,
            transform=transform,
            fill=-1,
            dtype=np.int16
        )

        # === Add region_code variable to dataset ===
        clipped["region_code"] = (("lat", "lon"), region_raster)

        # === Save output ===
        output_path = os.path.join(output_nc_folder, f"QLD_{nc_file}")
        clipped.to_netcdf(output_path)
        print(f"✅ Saved with region_code → {output_path}")

    except Exception as e:
        print(f"❌ Failed for {nc_file}: {e}")

# === Save region_code map as JSON ===
with open(os.path.join(output_nc_folder, "region_code_map.json"), "w") as f:
    json.dump(reverse_map, f, indent=2)
    print("✅ Saved region_code_map.json")



📂 Processing: 2000.radiation.nc
✅ Saved with region_code → Data - Modelling\Annual_DB\2. QLD_Cropped\Radiation\QLD_2000.radiation.nc

📂 Processing: 2001.radiation.nc
✅ Saved with region_code → Data - Modelling\Annual_DB\2. QLD_Cropped\Radiation\QLD_2001.radiation.nc

📂 Processing: 2002.radiation.nc
✅ Saved with region_code → Data - Modelling\Annual_DB\2. QLD_Cropped\Radiation\QLD_2002.radiation.nc

📂 Processing: 2003.radiation.nc
✅ Saved with region_code → Data - Modelling\Annual_DB\2. QLD_Cropped\Radiation\QLD_2003.radiation.nc

📂 Processing: 2004.radiation.nc
✅ Saved with region_code → Data - Modelling\Annual_DB\2. QLD_Cropped\Radiation\QLD_2004.radiation.nc

📂 Processing: 2005.radiation.nc
✅ Saved with region_code → Data - Modelling\Annual_DB\2. QLD_Cropped\Radiation\QLD_2005.radiation.nc

📂 Processing: 2006.radiation.nc
✅ Saved with region_code → Data - Modelling\Annual_DB\2. QLD_Cropped\Radiation\QLD_2006.radiation.nc

📂 Processing: 2007.radiation.nc
✅ Saved with region_code → Da